In [17]:
import os
import cv2
import pickle
from tqdm import tqdm
import numpy as np
import random

train_dir = 'data/train'
test_dir = 'data/test'

category_labels = {
    'Fighting': 0,
    'Shoplifting': 1,
    'Abuse': 2,
    'Arrest': 3,
    'Shooting': 4,
    'Robbery': 5,
    'Explosion': 6
}

def load_data(path_dir, labels):
    data = []
    
    for category, label in labels.items():
        category_dir = os.path.join(path_dir, category)

        if os.path.isdir(category_dir):
            for filename in tqdm(os.listdir(category_dir), desc=f"Loading {category}"):
                if filename.endswith(".jpg") or filename.endswith(".png"):
                    image_path = os.path.join(category_dir, filename)

                    try:
                        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                        image = cv2.resize(image, (50, 50))

                        # Reshape the image to 4D array (ImageDataGenerator requires 4D array)
                        image = image.reshape((1,) + image.shape + (1,))
                        data.append([image, label])
                    except Exception as e:
                        print(f"Error loading image {image_path}: {e}")

    return data

training_data = load_data(train_dir, category_labels)
test_data = load_data(test_dir, category_labels)

total_data = training_data + test_data
print(f"Loaded {len(total_data)} images")

Loading Explosion: 100%|█████████████████████████████████████████████████████████████████████| 6510/6510 [00:01<00:00, 4226.09it/s]


Loaded 189869 images


In [58]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers import LSTM, TimeDistributed, Conv1D, MaxPooling1D
from keras.layers import Dense, Dropout, Flatten
from keras.layers import concatenate, Input
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, CSVLogger
import time

images, labels = [], []

for image, label in total_data:
    images.append(image)
    labels.append(label)

images = np.array(images)
labels = np.array(labels)

# Reshape images for LSTM
images_lstm = images.reshape(images.shape[0], -1, 1)

# reshape for cnn
images = images.squeeze(axis=1)

# Set a seed for reproducibility
seed = 42

train_images_cnn, test_images_cnn, train_labels_cnn, test_labels_cnn = train_test_split(images, labels, test_size=0.1, random_state=seed)

train_images_lstm, test_images_lstm, train_labels_lstm, test_labels_lstm = train_test_split(images_lstm, labels, test_size=0.1, random_state=seed)

train_labels_cnn = to_categorical(train_labels_cnn, len(category_labels))
test_labels_cnn = to_categorical(test_labels_cnn, len(category_labels))

train_labels_lstm = to_categorical(train_labels_lstm, len(category_labels))
test_labels_lstm = to_categorical(test_labels_lstm, len(category_labels))

In [59]:
print(images.shape)
print(images_lstm.shape)

(189869, 50, 50, 1)
(189869, 2500, 1)


In [60]:
# CNN Model
cnn_input = Input(shape=(50, 50, 1))
model_CNN = Sequential()
model_CNN.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
model_CNN.add(LeakyReLU(negative_slope=0.1)) 
model_CNN.add(MaxPooling2D((2, 2), padding='same')) 
model_CNN.add(Dropout(0.25))
model_CNN.add(Conv2D(128, (3, 3), padding='same')) 
model_CNN.add(LeakyReLU(negative_slope=0.1)) 
model_CNN.add(MaxPooling2D(pool_size=(2, 2), padding='same')) 
model_CNN.add(Dropout(0.25))
model_CNN.add(Conv2D(256, (3, 3), padding='same'))
model_CNN.add(LeakyReLU(negative_slope=0.1))
model_CNN.add(MaxPooling2D(pool_size=(2, 2), padding='same')) 
model_CNN.add(Dropout(0.4))  
model_CNN.add(Flatten()) 
model_CNN.add(Dense(256)) 
model_CNN.add(LeakyReLU(negative_slope=0.1))            
model_CNN.add(Dropout(0.5))
model_CNN = model_CNN(cnn_input)

In [61]:
# LSTM Model
lstm_input = Input(shape=(2500,1))
model_lstm = Sequential()
model_lstm.add(LSTM(units = 8, return_sequences = True, activation='tanh'))
model_lstm.add(LSTM(units = 8, return_sequences = True))
model_lstm.add(Dense(4, activation='tanh'))
model_lstm.add(Dropout(0.2))
model_lstm.add(Flatten())
model_lstm = model_lstm(lstm_input)

<KerasTensor shape=(None, 10000), dtype=float32, sparse=False, ragged=False, name=keras_tensor_1335>

In [65]:
# Combine CNN and LSTM model
nb_classes = 7

combined = concatenate([model_CNN, model_lstm], axis=-1)
output = Dense(nb_classes, activation='softmax')(combined)
model_final = Model(inputs=[cnn_input, lstm_input], outputs=output)

model_final.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

csv_logger = CSVLogger('training.log', separator=',', append=False)
mc = ModelCheckpoint('CNN_LSTM.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

time1 = time.time()
history = model_final.fit([train_images_cnn, train_images_lstm], train_labels_lstm, batch_size=1000, epochs=20, validation_data=([test_images_cnn, test_images_lstm], test_labels_lstm), callbacks=[mc, csv_logger])
print (("Training time=", time.time()-time1))

np.save("CNN_LSTM_history.npy", history.history)

Epoch 1/20


2025-03-30 03:30:08.696146: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 963ms/step - accuracy: 0.3906 - loss: 3.4970
Epoch 1: val_loss improved from inf to 3.54009, saving model to CNN_LSTM.h5


171/171 ━━━━━━━━━━━━━━━━━━━━ 178s 998ms/step - accuracy: 0.3917 - loss: 3.4861 - val_accuracy: 0.6157 - val_loss: 3.5401
Epoch 2/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8788 - loss: 0.3887
Epoch 2: val_loss improved from 3.54009 to 0.52220, saving model to CNN_LSTM.h5


171/171 ━━━━━━━━━━━━━━━━━━━━ 196s 1s/step - accuracy: 0.8790 - loss: 0.3881 - val_accuracy: 0.8770 - val_loss: 0.5222
Epoch 3/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9634 - loss: 0.1257
Epoch 3: val_loss improved from 0.52220 to 0.07256, saving model to CNN_LSTM.h5


...

171/171 ━━━━━━━━━━━━━━━━━━━━ 223s 1s/step - accuracy: 0.9899 - loss: 0.0343 - val_accuracy: 0.9949 - val_loss: 0.0167
Epoch 7/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9913 - loss: 0.0283
Epoch 7: val_loss did not improve from 0.01666
171/171 ━━━━━━━━━━━━━━━━━━━━ 229s 1s/step - accuracy: 0.9913 - loss: 0.0283 - val_accuracy: 0.9950 - val_loss: 0.0173
Epoch 8/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9927 - loss: 0.0234
Epoch 8: val_loss improved from 0.01666 to 0.01390, saving model to CNN_LSTM.h5


171/171 ━━━━━━━━━━━━━━━━━━━━ 238s 1s/step - accuracy: 0.9927 - loss: 0.0234 - val_accuracy: 0.9954 - val_loss: 0.0139
Epoch 9/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9930 - loss: 0.0217
Epoch 9: val_loss improved from 0.01390 to 0.01241, saving model to CNN_LSTM.h5


171/171 ━━━━━━━━━━━━━━━━━━━━ 227s 1s/step - accuracy: 0.9930 - loss: 0.0217 - val_accuracy: 0.9960 - val_loss: 0.0124
Epoch 10/20
171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9941 - loss: 0.0182
Epoch 10: val_loss improved from 0.01241 to 0.01220, saving model to CNN_LSTM.h5


171/171 ━━━━━━━━━━━━━━━━━━━━ 223s 1s/step - accuracy: 0.9941 - loss: 0.0182 - val_accuracy: 0.9962 - val_loss: 0.0122
Epoch 11/20
  7/171 ━━━━━━━━━━━━━━━━━━━━ 3:14 1s/step - accuracy: 0.9958 - loss: 0.0143

KeyboardInterrupt: 

In [66]:
from keras.models import load_model
ems_model = load_model('./CNN_LSTM.h5')
# ems_model.summary() # summarize model.

loss, accuracy=ems_model.evaluate([test_images_cnn, test_images_lstm] ,test_labels_cnn) ## to get test accuracy and losses
print(loss, accuracy)

594/594 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - accuracy: 0.9958 - loss: 0.0140
0.012195613235235214 0.9962079524993896


In [69]:
time2 = time.time()
predict_prob = ems_model.predict([test_images_cnn, test_images_lstm])
y_pred = np.argmax(predict_prob, axis=1)
print('classification time:', time.time() - time2)

##print (y_pred)
y_true = np.argmax(test_labels_cnn, axis=1)

from sklearn.metrics import precision_recall_fscore_support as precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

precision = precision_score(y_true, y_pred, average='weighted')
print(f'Precision: {precision}')
recall = recall_score(y_true, y_pred, average='weighted')
print(f'Recall: {recall}')
f1 = f1_score(y_true, y_pred, average='weighted')
print(f'F1 score: {f1}')


test_eval = ems_model.evaluate([test_images_cnn, test_images_lstm], test_labels_cnn)

loss, accuracy = ems_model.evaluate([train_images_cnn, train_images_lstm], train_labels_cnn)
print('loss_train: ', loss, 'accuracy_train: ', accuracy)
print('Test loss:', test_eval[0], 'Test accuracy:', test_eval[1])

594/594 ━━━━━━━━━━━━━━━━━━━━ 35s 59ms/step
classification time: 35.18725514411926
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2532
           1       1.00      1.00      1.00      3310
           2       0.99      1.00      1.00      1962
           3       0.99      0.99      0.99      3002
           4       0.99      0.99      0.99      1459
           5       1.00      1.00      1.00      4132
           6       1.00      0.99      1.00      2590

    accuracy                           1.00     18987
   macro avg       1.00      1.00      1.00     18987
weighted avg       1.00      1.00      1.00     18987

Precision: (0.9962112389182382, 0.9962079317427713, 0.9962065999844298, None)
Recall: 0.9962079317427713
F1 score: 0.9962065999844298
594/594 ━━━━━━━━━━━━━━━━━━━━ 29s 48ms/step - accuracy: 0.9958 - loss: 0.0140
5341/5341 ━━━━━━━━━━━━━━━━━━━━ 260s 49ms/step - accuracy: 0.9975 - loss: 0.0076
loss_train:  0.007391293998807

In [71]:
import cv2
import numpy as np
import os

from keras.models import load_model
from PIL import Image

categories_labels = {'Fighting': 0, 'Shoplifting': 1, 'Abuse': 2, 'Arrest': 3, 'Shooting': 4, 'Robbery': 5, 'Explosion': 6}
labels_categories = {v: k for k, v in categories_labels.items()}  # reverse dictionary for label lookup

# Load the trained model
model = load_model('CNN_LSTM.h5')

def predict_image(image):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    image = cv2.resize(image, (50, 50))

    image_cnn = image.reshape((1,) + image.shape + (1,))
    image_lstm = image.reshape((1,) + (-1, 1))

    prediction = model.predict([image_cnn, image_lstm])

    label = np.argmax(prediction)

    return labels_categories[label]

# Test the function
image_path = "/Users/danliu/Desktop/SCR-20250330-epmr.png"  # Replace with your image path
print(predict_image(image_path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
Shoplifting
